In [1]:
import os
import networkx
import pickle
from namespace.graph.json_processor import get_data_matrix
from namespace.graph.comp_printer import print_comp

In [2]:
champion_matrix, trait_matrix, _ = get_data_matrix()

The graph `G` is a `networkx.DiGraph` object. Each of its edge points from a lower level comp to a higher level comp. We can both work from high level to low level or low level to high level. Since high level comps are more important for players, let's make a reverse of the graph.

In [3]:
with open('./simple_strategy_res/graph.pickle', 'rb') as f:
    G = pickle.load(f)
G_rev = G.reverse()
levels = [5, 6, 7, 8, 9]
subgraph_path = './subgraphs/'
roots = [node[0] for node in G_rev.nodes(data=True) if node[1]['layer'] == len(levels) - 1]
if not os.path.exists(subgraph_path):
    os.mkdir(subgraph_path)
for index, root in enumerate(roots):
    cur_G = networkx.DiGraph()
    champ_names, _, _ = print_comp(root, champion_matrix, trait_matrix)
    cur_G.add_node(str(champ_names), layer=len(levels) - 1)
    descendants = networkx.algorithms.dag.descendants(G_rev, root)
    for bit_code in descendants:
        champ_names, _, _ = print_comp(bit_code, champion_matrix, trait_matrix)
        cur_G.add_node(str(champ_names), layer=levels.index(len(champ_names)))
    for i in cur_G.nodes(data=True):
        for j in cur_G.nodes(data=True):
            if i[1]['layer'] - j[1]['layer'] == 1:
                cur_G.add_edge(i[0], j[0])
    networkx.write_graphml(cur_G, subgraph_path + f'tansition_{index}.graphml')

we save each subgraph to the folder `./subgraphs/` in `.graphml` format, each subgraph represent a transition tree, in this case from level 5 to level 9. Use Cytoscape to visualize it.

In [3]:
with open('./extract/level8.pickle', 'rb') as f:
    level8 = pickle.load(f)
for bit_code in level8:
    champ_names, active_traits, cost = print_comp(bit_code, champion_matrix, trait_matrix)
#     if "kha'zix" in  champ_names:
    print(champ_names)
    print(active_traits)
    print(cost)
    print()

['twitch', 'singed', 'mundo', 'lissandra', 'zac', 'urgot', 'yuumi', 'viktor']
['chemtech: 7', 'bruiser: 2', 'scholar: 2', 'cuddly: 1']
3.25

['twitch', 'singed', 'mundo', 'lissandra', 'zac', 'warwick', 'yuumi', 'viktor']
['chemtech: 7', 'bruiser: 2', 'scholar: 2', 'cuddly: 1']
3.0

['twitch', 'mundo', 'lissandra', 'zac', 'urgot', 'warwick', 'yuumi', 'viktor']
['chemtech: 7', 'bruiser: 2', 'scholar: 2', 'cuddly: 1']
3.38

['singed', 'mundo', 'lissandra', 'zac', 'urgot', 'warwick', 'yuumi', 'viktor']
['chemtech: 7', 'bruiser: 2', 'scholar: 2', 'cuddly: 1']
3.38

['twitch', 'singed', 'mundo', 'lissandra', 'zac', 'urgot', 'warwick', 'yuumi']
['chemtech: 7', 'bruiser: 2', 'scholar: 2', 'cuddly: 1']
2.88

['camile', 'twitch', 'singed', 'mundo', 'lissandra', 'zac', 'urgot', 'warwick']
['chemtech: 7', 'bruiser: 2', 'challenger: 2']
2.38

['talon', 'twitch', 'singed', 'mundo', 'lissandra', 'zac', 'urgot', 'warwick']
['chemtech: 7', 'assassin: 2', 'bruiser: 2']
2.5

['graves', 'twitch', 'singed'

['twitch', 'mundo', 'lissandra', 'zac', 'urgot', 'warwick', 'galio', 'viktor']
['chemtech: 7', 'socialite: 1', 'bruiser: 2']
3.38

['singed', 'mundo', 'lissandra', 'zac', 'urgot', 'warwick', 'galio', 'viktor']
['chemtech: 7', 'socialite: 1', 'bruiser: 2']
3.38

['twitch', 'singed', 'mundo', 'lissandra', 'urgot', 'warwick', 'yuumi', 'viktor']
['chemtech: 7', 'scholar: 2', 'cuddly: 1']
3.12

['twitch', 'singed', 'mundo', 'zac', 'urgot', 'warwick', 'yuumi', 'viktor']
['chemtech: 7', 'bruiser: 2', 'cuddly: 1']
3.12

['twitch', 'singed', 'lissandra', 'zac', 'urgot', 'warwick', 'yuumi', 'viktor']
['chemtech: 7', 'scholar: 2', 'cuddly: 1']
3.0

['twitch', 'singed', 'mundo', 'lissandra', 'zac', 'urgot', 'tahmkench', 'viktor']
['chemtech: 7', 'bruiser: 2', 'gluton: 1']
3.25

['twitch', 'singed', 'mundo', 'lissandra', 'zac', 'warwick', 'tahmkench', 'viktor']
['chemtech: 7', 'bruiser: 2', 'gluton: 1']
3.0

['twitch', 'singed', 'mundo', 'lissandra', 'urgot', 'warwick', 'tahmkench', 'viktor']
['che

['twitch', 'singed', 'twistedfate', 'mundo', 'lissandra', 'urgot', 'warwick', 'viktor']
['chemtech: 7', 'archanist: 2']
2.62

['twitch', 'singed', 'twistedfate', 'lissandra', 'zac', 'urgot', 'warwick', 'viktor']
['chemtech: 7', 'archanist: 2']
2.5

['twitch', 'singed', 'ekko', 'mundo', 'lissandra', 'urgot', 'warwick', 'viktor']
['chemtech: 7', 'assassin: 2']
2.88

['twitch', 'singed', 'ekko', 'lissandra', 'zac', 'urgot', 'warwick', 'viktor']
['chemtech: 7', 'assassin: 2']
2.75

['twitch', 'singed', 'poppy', 'mundo', 'zac', 'urgot', 'warwick', 'viktor']
['chemtech: 7', 'bruiser: 2']
2.62

['twitch', 'singed', 'ezreal', 'mundo', 'zac', 'urgot', 'warwick', 'viktor']
['chemtech: 7', 'bruiser: 2']
2.62

['twitch', 'singed', 'kassadin', 'mundo', 'zac', 'urgot', 'warwick', 'viktor']
['chemtech: 7', 'bruiser: 2']
2.62

['twitch', 'singed', 'lulu', 'mundo', 'zac', 'urgot', 'warwick', 'viktor']
['chemtech: 7', 'bruiser: 2']
2.75

['twitch', 'singed', 'zyra', 'mundo', 'lissandra', 'urgot', 'warwi